# Pixel Rearrangement Notebook

This notebook demonstrates how to rearrange the pixels of a source image so that they spatially resemble a target image while preserving the source colors. It also offers an interactive widget that animates the pixels moving from their original positions to the new arrangement in real time.

## How to use this notebook

1. Provide paths to your source and target images in the configuration cell near the bottom of the notebook.
2. Run the preprocessing cell to load and resize the images so that they share the same dimensions.
3. Run the rearrangement cell to compute the pixel mapping and display the source, target, and rearranged results.
4. Use the interactive widget to watch the pixels slide smoothly from their original layout to the new configuration. The play button and slider let you control the animation speed and position.

> Tip: For best performance in the animation, start with moderately sized images (e.g., heights between 120 and 240 pixels).

In [ ]:
from pathlib import Path
from typing import Dict, Tuple

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.colors import rgb_to_hsv
import ipywidgets as widgets

plt.rcParams["figure.facecolor"] = "white"


### Helper functions

In [ ]:

def load_and_prepare_images(
    source_path: Path,
    target_path: Path,
    output_height: int | None = 200,
    output_width: int | None = None,
) -> Tuple[np.ndarray, np.ndarray]:
    """Load two images, convert them to RGB, and resize them to a shared size."""
    source = Image.open(source_path).convert("RGB")
    target = Image.open(target_path).convert("RGB")

    if output_height is None and output_width is None:
        new_size = target.size
    else:
        if output_height is None:
            scale = output_width / target.width
            output_height = max(1, int(round(target.height * scale)))
        elif output_width is None:
            scale = output_height / target.height
            output_width = max(1, int(round(target.width * scale)))
        new_size = (int(output_width), int(output_height))

    source = source.resize(new_size, Image.LANCZOS)
    target = target.resize(new_size, Image.LANCZOS)
    return np.asarray(source), np.asarray(target)


def _sorted_indices_by_hsv(pixels: np.ndarray) -> np.ndarray:
    hsv = rgb_to_hsv(pixels.reshape(-1, 1, 3)).reshape(-1, 3)
    return np.lexsort((hsv[:, 2], hsv[:, 1], hsv[:, 0]))


def compute_pixel_mapping(
    source_img: np.ndarray,
    target_img: np.ndarray,
) -> Dict[str, np.ndarray]:
    if source_img.shape != target_img.shape:
        raise ValueError("Source and target images must share the same dimensions.")

    height, width, _ = source_img.shape
    source_pixels = source_img.reshape(-1, 3).astype(np.float32) / 255.0
    target_pixels = target_img.reshape(-1, 3).astype(np.float32) / 255.0

    source_order = _sorted_indices_by_hsv(source_pixels)
    target_order = _sorted_indices_by_hsv(target_pixels)

    rearranged_pixels = np.zeros_like(source_pixels)
    rearranged_pixels[target_order] = source_pixels[source_order]
    rearranged_img = (np.clip(rearranged_pixels, 0.0, 1.0) * 255).astype(np.uint8).reshape(height, width, 3)

    yy, xx = np.mgrid[0:height, 0:width]
    coords = np.stack([xx.reshape(-1), yy.reshape(-1)], axis=1).astype(np.float32)
    norm = np.array([max(width - 1, 1), max(height - 1, 1)], dtype=np.float32)
    normalized_coords = coords / norm

    return {
        "rearranged_img": rearranged_img,
        "source_coords": normalized_coords[source_order],
        "target_coords": normalized_coords[target_order],
        "colors": source_pixels[source_order],
        "shape": np.array([height, width], dtype=np.int32),
    }


def display_image_triplet(source_img: np.ndarray, target_img: np.ndarray, rearranged_img: np.ndarray) -> None:
    fig, axes = plt.subplots(1, 3, figsize=(12, 4))
    titles = ["Source", "Target", "Source colors rearranged"]
    for ax, img, title in zip(axes, [source_img, target_img, rearranged_img], titles):
        ax.imshow(img)
        ax.set_title(title)
        ax.axis("off")
    fig.tight_layout()


def _sample_for_animation(mapping: Dict[str, np.ndarray], max_pixels: int = 5000, seed: int = 0) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    coords0 = mapping["source_coords"]
    coords1 = mapping["target_coords"]
    colors = mapping["colors"]
    total = coords0.shape[0]

    if max_pixels is not None and total > max_pixels:
        rng = np.random.default_rng(seed)
        idx = rng.choice(total, size=max_pixels, replace=False)
        coords0 = coords0[idx]
        coords1 = coords1[idx]
        colors = colors[idx]

    return coords0, coords1, colors


def animate_pixel_flow(mapping: Dict[str, np.ndarray], max_pixels: int = 5000) -> None:
    coords0, coords1, colors = _sample_for_animation(mapping, max_pixels=max_pixels)

    fig, ax = plt.subplots(figsize=(5, 5))
    scatter = ax.scatter(coords0[:, 0], 1.0 - coords0[:, 1], c=colors, s=10, edgecolors="none")
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_title("Pixel rearrangement flow")

    def update(t: float = 0.0) -> None:
        interp = coords0 * (1.0 - t) + coords1 * t
        scatter.set_offsets(np.column_stack([interp[:, 0], 1.0 - interp[:, 1]]))
        fig.canvas.draw_idle()

    slider = widgets.FloatSlider(value=0.0, min=0.0, max=1.0, step=0.01, description="Blend")
    play = widgets.Play(interval=30, value=0.0, min=0.0, max=1.0, step=0.01)
    widgets.jslink((play, "value"), (slider, "value"))
    controls = widgets.HBox([play, slider])

    out = widgets.interactive_output(update, {"t": slider})
    display(controls, out)
    update(0.0)


### Configure image paths and preprocessing

Update the paths below to point at your source and target images. The images will be resized so they share the same dimensions before processing.

In [ ]:
SOURCE_PATH = Path("path/to/your/source_image.jpg")
TARGET_PATH = Path("path/to/your/target_image.jpg")

OUTPUT_HEIGHT = 180  # Adjust to control the resolution of the processing
OUTPUT_WIDTH = None  # Set to an integer to force a specific width


### Load, rearrange, and display results

Run the cell below after configuring the image paths. It loads the images, computes the mapping, visualizes the rearranged image, and builds the interactive animation widget.

In [ ]:

if not SOURCE_PATH.exists():
    raise FileNotFoundError(f"Source image not found: {SOURCE_PATH}")
if not TARGET_PATH.exists():
    raise FileNotFoundError(f"Target image not found: {TARGET_PATH}")

source_img, target_img = load_and_prepare_images(
    SOURCE_PATH,
    TARGET_PATH,
    output_height=OUTPUT_HEIGHT,
    output_width=OUTPUT_WIDTH,
)

mapping = compute_pixel_mapping(source_img, target_img)
rearranged_img = mapping["rearranged_img"]

display_image_triplet(source_img, target_img, rearranged_img)
animate_pixel_flow(mapping, max_pixels=5000)


### Optional: quick demo with synthetic images

The following cell generates two simple synthetic images to showcase the workflow without needing external files. Feel free to skip it if you plan to use your own images directly.

In [ ]:

from PIL import ImageDraw

height, width = 160, 160

# Create a colorful gradient source image
x = np.linspace(0, 1, width)
y = np.linspace(0, 1, height)
xx, yy = np.meshgrid(x, y)
source_demo = np.stack([
    (np.sin(2 * np.pi * xx) * 0.5 + 0.5),
    yy,
    (np.cos(2 * np.pi * yy) * 0.5 + 0.5)
], axis=-1)
source_demo_img = (source_demo * 255).astype(np.uint8)

# Create a target image with geometric shapes
img = Image.new("RGB", (width, height), color=(20, 20, 40))
draw = ImageDraw.Draw(img)
draw.rectangle([20, 20, 140, 140], outline=(240, 240, 240), width=3)
draw.ellipse([45, 45, 115, 115], fill=(180, 40, 60))
draw.rectangle([70, 115, 90, 135], fill=(255, 220, 120))
target_demo_img = np.asarray(img)

mapping_demo = compute_pixel_mapping(source_demo_img, target_demo_img)
display_image_triplet(source_demo_img, target_demo_img, mapping_demo["rearranged_img"])
animate_pixel_flow(mapping_demo, max_pixels=4000)
